In [1]:
import os
import os.path
import json
import sys
import pytesseract
import re
import difflib
import csv
import cv2
import dateutil.parser as dparser
from PIL import Image
import imutils
import numpy as np
import re

In [2]:
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 21))


In [3]:
Pan3 = cv2.imread("/home/hduser/Documents/Pancard/Pan4.jpg")
#Pan3 = cv2.cvtColor(Pan3, cv2.COLOR_BGR2GRAY)
print(Pan3.shape)

(298, 454, 3)


In [4]:
image = imutils.resize(Pan3, height=600)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKernel)
# cv2.imshow("Pan3_blackhat_image",blackhat)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [5]:
gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = (255 * ((gradX - minVal) / (maxVal - minVal))).astype("uint8")
# cv2.imshow("Pan3_gradX",gradX)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [6]:
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
# cv2.imshow("Pan3_thresh",thresh)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [7]:
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
thresh = cv2.erode(thresh, None, iterations=4)
# cv2.imshow("Pan3_thresh",thresh)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
# cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                        cv2.CHAIN_APPROX_SIMPLE)[-2]
info_list=[]

for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    pX = int((x + w) * 0.03)
    pY = int((y + h) * 0.03)
    (x, y) = (x - pX, y - pY)
    (w, h) = (w + (pX * 2), h + (pY * 2))
    roi = image[y:y + h, x:x + w].copy()
     # roi=cv2.resize(roi,(650,500))
    #print(roi.shape)
    image=cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _,img = cv2.threshold(gray,100,255,cv2.THRESH_BINARY)
    img=cv2.resize(img,(590,600))
    im=pytesseract.image_to_string(img)
    im=re.split(r'\n',im)
    
print(im)
# for pan in pan:
#     if re.search(pan, im):
#         print(im)

    
# show the output images
# cv2.imshow("Image", image)
# cv2.imshow("ROI", roi)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


['"SW Hum cggg‘u Hm (1 Hum!', '', 'INCUMETAXDEPARTMEM J‘b‘ GOVT.\\0FINDIA', 'SHRIJEET HAJAN SALAGARE s"? \' "5,1', 'RAJAN BHIKU SALAGARE‘', '', 'RIM/1934.', 'Permanent Account Number', '', 'CWEPS5009R', '', 'Signature', '', ' ', '', '2807201 0']


In [9]:
im1=' '.join(im)    #convert list to string
im1

'"SW Hum cggg‘u Hm (1 Hum!  INCUMETAXDEPARTMEM J‘b‘ GOVT.\\0FINDIA SHRIJEET HAJAN SALAGARE s"? \' "5,1 RAJAN BHIKU SALAGARE‘  RIM/1934. Permanent Account Number  CWEPS5009R  Signature     2807201 0'

In [20]:
import re 
import Levenshtein
match_string = "Permanent Account Number"
substring="Permanent Account Number"
match = re.findall(r'[A-Z]{5}[0-9][0-9][0-9][0-9][A-Z]', im1)
if substring in im1:
    if(Levenshtein.ratio(match_string,substring)) > 0.5:
        print("Levenshtein Distance:",Levenshtein.ratio(match_string, substring))
        print("Pan Number found..")
        print("Permanent Account Number ",match)
    else:
        print("Please upload clear pan image")
        

Levenshtein Distance: 1.0
Pan Number found..
Permanent Account Number  ['CWEPS5009R']


In [10]:
# def Pan_num_extract(value, a, b):
#     # Find and validate before-part.
#     pos_a = value.find(a)
#     if pos_a == -1: return ""
#     # Find and validate after part.
#     pos_b = value.rfind(b)
#     if pos_b == -1: return ""
#     # Return middle part.
#     adjusted_pos_a = pos_a + len(a)
#     if adjusted_pos_a >= pos_b: return ""
#     return value[adjusted_pos_a:pos_b]

In [11]:
# Pan_num = Pan_num_extract(im1, "Permanent", "Signature")

In [13]:
# import re 
# re.search(r'Number: [0-6]', 'Number: 5').group()
# phoneNumRegex = re.compile(r'[A-z]\d\d\d\d-[A-Z]') 
# print(phoneNumRegex)
# # mo = phoneNumRegex.search('My number is 415-555-4242.') 
# # print('Phone number found: ' + mo.group())

In [15]:
# import re
# foo = 'CWEPS5009R '
# match = re.findall(r'[A-Z]{5}[0-9][0-9][0-9][0-9][A-Z]', foo)
# print(match)

In [ ]:
# import Levenshtein
# substring = "Permanent Account Numbr"
# for i in im:
#     for substring in i:
#         print ("Your substring was found!")
        
# #         if (Levenshtein.ratio(substring,i)) > 0.5:
# #                 print("Levenshtein Distance:",Levenshtein.ratio(substring,i))
# #                 print("Pan Number found..")
# #                 print("Permanent",Pan_num) 
# #         else:
# #                 print("Levenshtein Distance:",Levenshtein.ratio(substring,i))
# #                 print("Please upload clear pan image")

In [ ]:
# import Levenshtein
# match_string = "Permanent Account Number"
# for i in im:
#     print (Levenshtein.ratio(match_string, i))
#     if (Levenshtein.ratio(match_string, i)) > 0.5:
#                 print("Levenshtein Distance:",Levenshtein.ratio(match_string, i))
#                 print("Pan Number found..")
#                 print("Permanent",Pan_num)
#     else:
#                  print('No')


In [ ]:
# all_text ="seema"

# if any(word in all_text for word in im):
#     if (Levenshtein.ratio(match_string, i)) > 0.5:
#             print("Levenshtein Distance:",Levenshtein.ratio(match_string, i))
#             print("Pan Number found..")
#             print("Permanent",Pan_num)
#     else:
#          print("no")


In [ ]:
# # match = "Permanent Account Number"
# import Levenshtein
# match="Permanent Acocunt Number"

# if (Levenshtein.ratio(match, i)) > 0.5:
#     for match in im:
#             print("Levenshtein Distance:",Levenshtein.ratio(match,i))
#             print("Pan Number found..")
#             print("Permanent",Pan_num)
# else:
#            print("Please upload clear pan image")
        

In [ ]:
# import Levenshtein

# match="Permanent Accout Number"
# for match in im:
#         if (Levenshtein.ratio(match, i)) > 0.5:
#             print("Levenshtein Distance:",Levenshtein.ratio(match,i))
#             print("Pan Number found..")
#             print("Permanent",Pan_num)
#             break
#         else:
#             print("no")
            
         


In [ ]:
# import Levenshtein
# match="Permanent Account Number"
# for i in im1:
#     for match in i:
#         x= Levenshtein.ratio(match,i)
#         if x > 0.5:
#             print("Done")
#         elif x < 0.5:
#             print("JI")


In [ ]:
# import Levenshtein
# match_string = "Permanent Account Number"

# for match in im1:
#     print("Found")
#     x=Levenshtein.ratio(match_string,match)
#     print(im1)
#     print(x)
#     if  x > 0.5:
#         print("Levenshtein Distance:",x)
#         print("Pan Number found..")
#         print("Permanent",Pan_num)
#         break
#     elif x < 0.5:
#         print("Please upload clear pan image")
#         break
#     else:
#         print("Please upload clear pan image")

In [ ]:
# ASCIIcodes = []
# match_string = "Permanent Account Number"
# for mammal in im:
#     for match_string in mammal:
#         x=Levenshtein.ratio(match_string,match)
#         if  x > 0.5:
# #             print("Levenshtein Distance:",x)
# #             print("Pan Number found..")
# #             print("Permanent",Pan_num)
#             ASCIIcodes.append(Pan_num)
#     for match_string not in mammal:
#         print("NO")
           
        
# print(ASCIIcodes)
# print("Levenshtein Distance:",x)
# print("Pan Number found..")

In [ ]:
# x = "Permanent Account Number"

# # list = [10, 20, 30, 40, 50 ]; 
  
# # if ( x not in im ): 
# #    print ("x is NOT present in given list")
# # # else: 
# # #    print ("x is  present in given list")

# for x in im :
#         if(Levenshtein.ratio(x, i)) > 0.5: 
#              print ("y is present in given list")
#              print("Levenshtein Distance:",Levenshtein.ratio(x, i))
#              print("permanent",Pan_num)
#         else: 
#              print ("y is NOT present in given list")
#              print("Levenshtein Distance:",Levenshtein.ratio(x, i))


In [ ]:
# import Levenshtein
# match_string = "Permanent Account Number"
# for i in im1:
#     ratio=Levenshtein.ratio(match_string,i)
#     print(ratio)
#     if ratio > 0.19:
#         print("Pan Number found:")
#         print(Pan_num)
#     else:
#         print("Please upload clear pan image")

In [ ]:
# one = im[6]
# two=im[2][0:24]
# three=im[3]
# print("Pan_number:",one)
# print("Name:",two)
# print("Father Name:",three)


In [ ]:

# import Levenshtein
# match_string = "Permanent Account Number"
# for i in im:
# #     print (Levenshtein.ratio('Permanent Account Number', i))
#     if (Levenshtein.ratio(match_string, i)) > 0.5:
#                 print("Levenshtein Distance:",Levenshtein.ratio(match_string, i))
#                 print("Pan Number found..")
#                 print("Permanent",Pan_num)
#     else:
#                  print('No')


In [ ]:
# if bad_chars in im:
#     print ("Echipele nu sunt bune")  
# else:
#     print("NO")

In [ ]:
# sub = 'GOVT.\\0FINDIA'

# for text in im:
#     if sub in text:
#         print(text)

In [ ]:
# im=str(im).strip('[]')
# start = '0FINDIA'
# end = 'Signature'
# final_String= im.split(start)[1].split(end)[0]
# final_String

In [ ]:
# for im in final_String.split("\n"):
#     im=re.sub(r"[^a-zA-Z0-9]+", '\n', im)
#     print(im)

In [ ]:
# import Levenshtein
# Levenshtein.ratio('hello', 'hello')